# Translation


* Translation is the shifting of an image along the x and y axis.
* Using translation, we can shift an image up, down, left, or right, along with any combination of the above.

Mathematically, we define a translation matrix M that we can use to translate an image:

M = $\begin{bmatrix}1 & 0 & t_{x} \\   0 & 1 & t_{y}\end{bmatrix}$

In [1]:
# import the necessary packages
import numpy as np
import argparse
import cv2

# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--image", required=True, help="Path to the image")
args = vars(ap.parse_args())

# load the image and show it
image = cv2.imread(args["image"])
cv2.imshow("Original", image)

# NOTE: Translating (shifting) an image is given by a NumPy matrix in
# the form:
#	[[1, 0, shiftX], [0, 1, shiftY]]
# You simply need to specify how many pixels you want to shift the image
# in the X and Y direction -- let's translate the image 25 pixels to the
# right and 50 pixels down
M = np.float32([[1, 0, 25], [0, 1, 50]])
shifted = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))
cv2.imshow("Shifted Down and Right", shifted)

# now, let's shift the image 50 pixels to the left and 90 pixels up, we
# accomplish this using negative values
M = np.float32([[1, 0, -50], [0, 1, -90]])
shifted = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))
cv2.imshow("Shifted Up and Left", shifted)


usage: __main__.py [-h] -i IMAGE
__main__.py: error: the following arguments are required: -i/--image


SystemExit: 2

/Users/soumendra/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
def translate(image, x, y):
    # define the translation matrix and perform the translation
    M = np.float32([[1, 0, x], [0, 1, y]])
    shifted = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))

    # return the translated image
    return shifted

# What is an Affine Transformation?

It is any transformation that can be expressed in the form of a matrix multiplication (linear transformation) followed by a vector addition (translation).

We can use an Affine Transformation to express:

1. Rotations (linear transformation)
2. Translations (vector addition)
3. Scale operations (linear transformation)

In essence, an Affine Transformation represents a relation between two images.

The usual way to represent an Affine Transform is by using a $2 \times 3$ matrix.

![](./images/lec01.png)

![](./images/lec02.png)


# Rotation

![](./images/lec03.png)

Given an (x, y)-Cartesian plane, this matrix can be used to rotate a vector $\theta$ degrees (counter-clockwise) about the origin. In this case, the origin is normally the center of the image; however, in practice we can define any arbitrary (x, y) coordinate as our rotation center.

From the original image I, the rotated image R is then obtained by simple matrix multiplication: R = IM

However, OpenCV also provides the ability to 
* scale (resize) an image and 
* provide an arbitrary rotation center to perform the rotation about.

Our modified rotation matrix M is thus:

![](./images/lec04.png)

Where:

$\alpha = scale * cos \theta$ and $\beta = scale * sin \theta$ and $c_{x}$ and $c_{y}$ are the respective (x, y)-coordinates that the rotation is performed about.

In [ ]:
# import the necessary packages
import numpy as np
import argparse
import cv2

# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--image", required=True, help="Path to the image")
args = vars(ap.parse_args())

# load the image and show it
image = cv2.imread(args["image"])
cv2.imshow("Original", image)

# grab the dimensions of the image and calculate the center of the image
(h, w) = image.shape[:2]
(cX, cY) = (w / 2, h / 2)

# rotate our image by 45 degrees
M = cv2.getRotationMatrix2D((cX, cY), 45, 1.0)
rotated = cv2.warpAffine(image, M, (w, h))
cv2.imshow("Rotated by 45 Degrees", rotated)

# rotate our image by -90 degrees
M = cv2.getRotationMatrix2D((cX, cY), -90, 1.0)
rotated = cv2.warpAffine(image, M, (w, h))
cv2.imshow("Rotated by -90 Degrees", rotated)


Just as we defined a matrix to translate an image, we also define a matrix to rotate the image. Instead of manually constructing the matrix using NumPy (which can be a bit tedious), we’ll just make a call to the cv2.getRotationMatrix2D method .

The cv2.getRotationMatrix2D  function takes three arguments.

* The first argument is the point in which we want to rotate the image about.
* We then specify $\theta$, the number of (counter-clockwise) degrees we are going to rotate the image by.
* The last argument is the scale of the image. Here you can specify a floating point value, where 1.0 means the same, original dimensions of the image are used.
    - If you specified a value of 2.0, the image would be doubled in size.
    - A value of 0.5 halve the size of the image.
    
Once we have our rotation matrix M from the cv2.getRotationMatrix2D  function, we can apply the rotation to our image using the cv2.warpAffine method.

In [ ]:
def rotate(image, angle, center=None, scale=1.0):
    # grab the dimensions of the image
    (h, w) = image.shape[:2]

    # if the center is None, initialize it as the center of
    # the image
    if center is None:
        center = (w // 2, h // 2)

    # perform the rotation
    M = cv2.getRotationMatrix2D(center, angle, scale)
    rotated = cv2.warpAffine(image, M, (w, h))

    # return the rotated image
    return rotated

# Resizing

Decreasing vs increasing size - interpolation

In [ ]:
def resize(image, width=None, height=None, inter=cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation=inter)

    # return the resized image
    return resized